In [5]:
import requests
import pandas as pd, json_normalize

In [ ]:
headers = {'Content-Type': 'application/json'}

In [ ]:
host  = ''
port = ''
user = ''
password = ''

In [ ]:
# use wild cards as needed myindex* or error_log*
index = ''

# per request how many records you want to fetch, where 10000 is maximum / fetch
size = 10000

# fields that you want to the fetch to be limited to. There might be many fields and you only
# want to fetch one field, so it would be good to define it to reduce data load
fields = ['message']

In [ ]:
# scroll=1m is a timeout as 1 milliseconds
first_call_url = f'http://{user}:{password}@{host}:{port}/{index}/_search?scroll=1m'

In [ ]:
# all subsequent calls after the first call would use this URL to paginate
subsequent_call_url = f'http://{user}:{password}@{host}:{port}/_search/scroll/'

In [246]:
first_call_url_payload = {
    "size": 10000,
    "query": { 
        "match_all": {
            
        } 
    },
    {
      "range": {
        "@timestamp": {
          "gte": "YYYY-MM-DDTHH:MM:SS.000Z",
          "lte": "YYYY-MM-DDTHH:MM:SS.000Z",
          "format": "strict_date_optional_time"
        }
      }
    }
    "_source": [fields]
}

In [247]:
# This is the first request
res = requests.post(first_call_url, json=first_call_url_payload, headers=headers)

In [248]:
# This has the result document that we want to transform into a dataframe for easy operations
query_data = res.json()['hits']['hits']

# This has the scroll id for the next call so that we fetch next batch of data
scroll_id = res.json()['_scroll_id']

In [249]:
# Can use json_normalize instead of this loop options 
df = pd.DataFrame([ x['_source'] for x in query_data])

In [251]:
# preparing payload for next call
scroll_payload = { 'scroll': '5s', 'scroll_id': scroll_id }

In [252]:
$# keep looping until in response there is data
while data:
    res = requests.post(next_call_url, json=scroll_payload, headers=headers)
    data = res.json()['hits']['hits']
    tmp = pd.DataFrame([ x['_source'] for x in res.json()['hits']['hits']])

    # concatinating the data 
    df = pd.concat([df, tmp])
    scroll_id = res.json()['_scroll_id']
    payload = { 'scroll': '1m', 'scroll_id': scroll_id }

In [ ]:
# this dataframe frame has entire data
df

# if your dataframe has a single column name messge and it has text string with comma separated
# values and you have to create columns out of it, you can do something like this

df[['a', 'b', 'c']] = df.message.str.split(',',expand=True)